
    
    
## <H2 align="center"> <font color='white'> **MAF1731B**: Microstructure and Trading Systems </H2>

### <H3 align="center"> <font color='white'> **Laboratory 4**: Market Microstructure </H3>

 <H5 align="center"> <em> Professor: Villalobos Ramos Omar Antonio <em>

 <p align="center"> <img style=" margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="350" height="150" /> 

 <strong> <H5 align="left"> Student:
Esteban Marquez Delgado <strong>

 *Registry*:
if700637: https://github.com/EstebanMqz/MyST_LAB_4

 <center> <font color= #555555> <font size = 4> November 2022 </a> <font color= #555555> <font size = 4> </a></font>

<hr style="border:0.02in solid gray"> </hr>

<center> <font color= #555555> <font size = 6> Laboratory 4 </font> <br> <br> <font color= #555555> <font size = 5> Market Microstructure  </font>

<hr style="border:0.02in solid gray"> </hr>

### <font color= #555555> <font size = 4> Abstract </font>

This document was prepared as delivery for Laboratory 4 of Microstructure and Trading Systems - MAF1731B class during the Autumn 2022 course at ITESO for the Bachelor of Financial Engineering. In order to analyze the market microstructure in criptocurrencies markets containing the following 3 sections:

1. **Consume CCXT data**: Download Criptocurrencies Order Books from exchanges (kraken, ftx, currencycom, coinmate, etc).

2. **Microstructure Visualization**: Build Time Series using the Order Books obtained in the previous point from the following elements of the Market Microstructure for each asset and exchange (Levels, Bid Volume, Ask Volume, Total Volume, Mid Price, VWAP.)

3. **Microstructure Modeling**: Estimation of Roll Effective Spread using the lagged series at t-5.



<font color= lightblue> <font size = 4> 0. Libraries and Dependencies: </font>

<font color= gray>

**Note**: To run this notebook it is necessary to have the following libraries contained in the [requirements.txt](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/requirements.txt) script of this project.

To install the libraries in the script run the following cell:

In [ ]:
!pip install -r requirements.txt

If you prefer, manual installation can be done with the individual installation of the following libraries:

<font color= lightblue> <font size = 1> 
+ pandas >= 1.3.4
+ numpy >= 1.19.1
+ jupyter >= 1.0.0
+ chart_studio >= 1.1
+ plotly >= 4.14
+ datetime >= 4.7
+ pandas_datareader >= 0.10.0
+ tk >= 0.10
+ plotly >= 5.60
+ yahoofinancials >= 1.60
+ scipy >= 1.7.3
+ regex >= 2022.3.15
+ matplotlib >= 3.5.1
+ fire >= 0.4.0
+ ccxt >= 2.1.33
+ time >= 3.11 

### <font color= lightblue> <font size = 4> 0.1 Introduction: </font>
In this laboratory we will use information from the Cryptocurrency Order Book of various international exchanges, with the primary objective of visualizing and modeling the microstructure of the market through the behavior of the Order Book and its characteristics such as; Matching Engine, Levels, Ask-Bid Volumes, etc.
<font color= gray>




##### *General help:*
In order, to retrieve specific help from the scripts tree in the project execute the help function on the script or in specific modules from the following:
+ [data.py](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/data.py)
+ [functions.py](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/functions.py)
+ [visualizations.py](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/visualizations.py)

In [1]:
## Libraries
import pandas as pd # Analysis and data management
import pandas_datareader as pdr
import pandas_datareader.data as web #Data extraction from the web
import numpy as np
from datetime import datetime, timedelta, date
from tkinter.ttk import Style
from tkinter import Y
import plotly.graph_objects as go #Plotly
import plotly.express as px
import warnings
from yahoofinancials import YahooFinancials # Yfinance 
import yfinance as yf
import ccxt #Criptocurrencies
from scipy.optimize import minimize # Optimization 
from collections import Counter #Counter
import time
import re
import io 
import glob
import os
#.Py archives in repo
import functions as fn 
import visualizations as vs
import data as dt
from os import path
import fire
%matplotlib inline

<font color= orange> <font size = 5> 1. Consume CCXT data: </font>

<font color= lightblue> <font size = 4> 1.1 Levels of Order Book (Bids/Asks): </font>

+ <font color= lightgray> <font size = 2> *Bids*: </font>

In [20]:
exchanges= ['binance', 'bitfinex','kraken']
criptos = ['BTC/USDT', 'ETH/USDT', 'XRP/USDT']

binance = ccxt.binance()
time_stamp = binance.iso8601(binance.milliseconds())
bi_btc_ob = binance.fetch_order_book('BTC/USDT', limit=25) #Order Books 
bi_btc_ob_bid = pd.DataFrame(bi_btc_ob['bids'], columns = ['price','quantity']) #Levels and Qt
bi_btc_ob_ask = pd.DataFrame(bi_btc_ob['asks'], columns = ['price','quantity']) 


bid = bi_btc_ob['bids'][0][0] if len (bi_btc_ob['bids']) > 0 else None
ask = bi_btc_ob['asks'][0][0] if len (bi_btc_ob['asks']) > 0 else None
spread = (ask - bid) if (bid and ask) else None
#pd.DataFrame(bid)
bi_btc_ob_bid.head()

,price,quantity
0,20708.15,0.01188
1,20708.14,0.00408
2,20707.85,0.02294
3,20707.84,0.11030
4,20707.83,0.02933


+ <font color= lightgray> <font size = 2> *Asks*: </font>

In [8]:
bi_btc_ob_ask.head()

,price,quantity
0,20716.30,0.00970
1,20716.32,0.05023
2,20716.34,0.01207
3,20716.38,0.04000
4,20716.39,0.05048


<font color= lightblue> <font size = 4> 1.2 OHLCV: </font>

In [13]:
cols_ohlcv = ['exchange','timestamp', 'open', 'high', 'low', 'close', 'volume']
ohlcv=binance.fetch_ohlcv('BTC/USDT', limit=1)
ohlcv

ohlcv_df=pd.DataFrame(ohlcv, columns=cols_ohlcv[1:])
ohlcv_df['timestamp']=pd.to_datetime(ohlcv_df['timestamp'])
ohlcv_df
        

,timestamp,open,high,low,close,volume
0,1970-01-01 00:27:47.845920,20717.96,20730.2,20716.31,20726.96,160.22794


<font color= lightblue> <font size = 4> 1.3 OB metrics: </font>

In [10]:
binance = ccxt.binance()

ob = binance.fetch_order_book(str(criptos[0]), limit=25)

bids = pd.DataFrame(ob['bids'], columns = ['price','quantity'])
asks = pd.DataFrame(ob['asks'], columns = ['price','quantity'])

time_stamp = binance.iso8601(binance.milliseconds())  
levels = asks.count()[0]
ask_volume = asks.quantity.sum()
bid_volume = bids.quantity.sum()
total_volume = bid_volume + ask_volume
mid_price = (bids.price[0] + asks.price[0])*.5
vwap = bids.price[0] + asks.price[0] / (bid_volume + ask_volume)

ob= pd.DataFrame({"exchange" : exchanges[0],"timeStamp" : time_stamp, "level" : levels,
                 'ask_volume':  ask_volume, 'bid_volume': bid_volume, 'total_volume': total_volume,
                 'mid_price': mid_price, 'vwap' : vwap}, index = ['0'])

ob.index.name = 'Order_Book' 

ob

,exchange,timeStamp,level,ask_volume,bid_volume,total_volume,mid_price,vwap
Order_Book,,,,,,,,
0,binance,2022-11-07T18:29:53.355Z,25,1.02415,1.78997,2.81412,20725.015,28089.502797


+ <font color= lightgray> <font size = 2> *OB Visualization*: </font>

In [19]:
binance = ccxt.binance()
time_stamp = binance.iso8601(binance.milliseconds())
bi_btc_ob = binance.fetch_order_book('BTC/USDT', limit=30) #Order Books 

bi_btc_ob_bid = pd.DataFrame(bi_btc_ob['bids'], columns = ['price','quantity']) #Levels and Qt
bi_btc_ob_ask = pd.DataFrame(bi_btc_ob['asks'], columns = ['price','quantity']) 


bid = bi_btc_ob['bids'][0][0] if len (bi_btc_ob['bids']) > 0 else None
ask = bi_btc_ob['asks'][0][0] if len (bi_btc_ob['asks']) > 0 else None
spread = (ask - bid) if (bid and ask) else None

fig = go.Figure(layout_xaxis_range=[0,10])

fig.add_trace(go.Bar(
    y= np.array(bi_btc_ob_ask.price.astype(str)),
    x= bi_btc_ob_ask.quantity,
    orientation='h',
))

fig.add_trace(go.Bar(
    y= np.array(bi_btc_ob_bid.price.astype(str)),
    x= bi_btc_ob_bid.quantity,
    orientation='h',
))
fig.update_layout(title_text="OB Asks vs Bids. Mid-Price: "+str((round(bid+ask,6))*.5),
                  title_font_size=15,)   
fig.show()

<font color= orange> <font size = 5> 2. Microstructure Visualization </font>

<font color= lightblue> <font size = 4> 2.1 Dataframes </font>

In [23]:
#exchanges=[exchanges[i]+str('()') for i in range(0, len(exchanges))]

cols_ob = ['exchange', 'timestamp', 'level', 'ask_volume', 'bid_volume', 'total_volume', 'mid_price', 'vwap']
ob_df = []
cols_ohlcv = ['exchange','timestamp', 'open', 'high', 'low', 'close', 'volume']
ohlcv_df = []
#for i in range(5):
for i in range(100):
    binance = ccxt.binance()
    ob = binance.fetch_order_book(str(criptos[0]), limit=25)
    ohlcv=binance.fetch_ohlcv('BTC/USDT', limit=1)

    #Order Book
    bids = pd.DataFrame(ob['bids'], columns = ['price','quantity'])
    asks = pd.DataFrame(ob['asks'], columns = ['price','quantity'])
    timestamp = binance.iso8601(binance.milliseconds())
    levels = asks.count()[0]
    ask_volume = asks.quantity.sum()
    bid_volume = bids.quantity.sum()
    total_volume = bid_volume + ask_volume
    mid_price = (bids.price[0] + asks.price[0])*.5
    vwap = bids.price[0] + asks.price[0] / (bid_volume + ask_volume)

    #OHLCV
    Op = ohlcv[0][1]
    Hi = ohlcv[0][2]
    Lo = ohlcv[0][3]
    Cl = ohlcv[0][4]
    Vol = ohlcv[0][5]

    #Append OB data to df.
    ob_df.append([exchanges[0], timestamp, levels, ask_volume, bid_volume, ask_volume, mid_price, vwap])
    df1 = pd.DataFrame(ob_df, columns=cols_ob)
    df1.index.name = 'Order_Books'

    #OHLCV
    #Append OHLCV data to df.
    ohlcv_df.append([exchanges[0], timestamp,Op, Hi, Lo, Cl, Vol])
    df2 = pd.DataFrame(ohlcv_df, columns=cols_ohlcv)
    df2.index.name = 'OHLCV'
    #timestamp = binance.iso8601(binance.milliseconds())
    #df2['timestamp'] = timestamp

    time.sleep(1)
df1['timestamp'] = [df1['timestamp'][i].strip('Z') for i in range(0, len(df1))]
df2['timestamp'] = [df2['timestamp'][i].strip('Z') for i in range(0, len(df2))]
        
display(df1)
display(df2)



,exchange,timestamp,level,ask_volume,bid_volume,total_volume,mid_price,vwap
Order_Books,,,,,,,,
0,binance,2022-11-07T18:40:21.074,25,1.47157,2.37442,1.47157,20713.410,26098.955834
1,binance,2022-11-07T18:40:23.503,25,1.16049,5.00277,1.16049,20712.365,24072.753026
2,binance,2022-11-07T18:40:25.884,25,0.88240,5.60663,0.88240,20712.865,23904.673290
3,binance,2022-11-07T18:40:28.367,25,2.10579,5.23018,2.10579,20709.180,23531.651806
4,binance,2022-11-07T18:40:31.769,25,1.34400,2.83071,1.34400,20708.785,25669.047624
5,binance,2022-11-07T18:40:34.839,25,0.87804,6.20227,0.87804,20709.055,23633.793006
6,binance,2022-11-07T18:40:38.788,25,0.51684,2.52070,0.51684,20709.280,27526.858942
7,binance,2022-11-07T18:40:41.457,25,1.81191,1.40217,1.81191,20708.725,27151.758807
8,binance,2022-11-07T18:40:44.147,25,2.95369,1.08793,2.95369,20706.690,25829.843178


,exchange,timestamp,open,high,low,close,volume
OHLCV,,,,,,,
0,binance,2022-11-07T18:40:21.074,20718.26,20718.69,20712.00,20713.29,76.09383
1,binance,2022-11-07T18:40:23.503,20718.26,20718.69,20712.00,20712.05,79.69844
2,binance,2022-11-07T18:40:25.884,20718.26,20718.69,20711.36,20713.07,84.75890
3,binance,2022-11-07T18:40:28.367,20718.26,20718.69,20707.76,20708.49,109.76420
4,binance,2022-11-07T18:40:31.769,20718.26,20718.69,20707.76,20709.14,115.00128
5,binance,2022-11-07T18:40:34.839,20718.26,20718.69,20707.76,20708.76,117.92094
6,binance,2022-11-07T18:40:38.788,20718.26,20718.69,20707.76,20708.42,131.59485
7,binance,2022-11-07T18:40:41.457,20718.26,20718.69,20707.76,20708.60,134.85469
8,binance,2022-11-07T18:40:44.147,20718.26,20718.69,20706.42,20706.86,143.34072


<font color= orange> <font size = 5> 3. Microstructure Modelling: </font>

### <font color= orange> <font size = 5> 4. Bibliography: </font>

<font color= lightgray>

+ Villalobos, O, 2021. **Python Project Template**. Available at: https://github.com/OmarVillalobos/python-project. 
(Accessed: November 5, 2022). 

+ Financial Engineering, **Marketmicrostructure.pptx**. ITESO -  Microstructure and Trading Systems (ITE1731 - DMAF). 
Available at: https://docs.google.com/presentation/d/1uYyMERKKGeZ0MPqbJ3bcir9lfpzbeAH9ZMmxbYysokQ/edit?usp=sharing (Accessed: November 5, 2022). 

+ Muñoz E., J **Roll Model** *(spanish)*. ITESO - (ITE1731 - DMAF) Available at: https://github.com/EstebanMqz/MyST_LAB_4/blob/main/Roll%20.pdf (Accessed: November 5, 2022).

+ Muñoz E., J **Asset Pricing Theory** *(spanish)*. ITESO - (ITE1731 - DMAF) Available at: https://github.com/EstebanMqz/MyST_LAB_4/blob/main/APT.pdf (Accessed: November 5, 2022). 